# Visualizing school workforce in England
## From the [MakeoverMonday](https://www.makeovermonday.co.uk/#) prompt for the week of 2020-9-14
- Description of the data can be found on [uk's goverment website](https://explore-education-statistics.service.gov.uk/find-statistics/school-workforce-in-england#releaseHeadlines-charts) 
- The specific data file used here can be downloaded from [data.world](https://data.world/makeovermonday/2020w37-school-workforce-in-england)

In [1]:
import numpy as np
import pandas as pd

import altair as alt

In [2]:
# import the data
dat = pd.read_csv('data-school-workforce-in-england.csv')
dat
# I did some data checks:
# some columns have no variation: location, school_type, age_category

location location_code geographic_level  time_period        gender  \
0    England     E92000001          country       201011        Female   
1    England     E92000001          country       201011        Female   
2    England     E92000001          country       201011        Female   
3    England     E92000001          country       201011        Female   
4    England     E92000001          country       201011          Male   
..       ...           ...              ...          ...           ...   
155  England     E92000001          country       201920         Total   
156  England     E92000001          country       201920  Unclassified   
157  England     E92000001          country       201920  Unclassified   
158  England     E92000001          country       201920  Unclassified   
159  England     E92000001          country       201920  Unclassified   

                    school_type                      grade age_category  \
0    Total state-funded schools         Classroom teachers        Total   
1    Total state-funded schools              Head teachers        Total   
2    Total state-funded schools  Other Leadership teachers        Total   
3    Total state-funded schools                      Total        Total   
4    Total state-funded schools         Classroom teachers        Total   
..                          ...                        ...          ...   
155  Total state-funded schools                      Total        Total   
156  Total state-funded schools         Classroom teachers        Total   
157  Total state-funded schools              Head teachers        Total   
158  Total state-funded schools  Other Leadership teachers        Total   
159  Total state-funded schools                      Total        Total   

    average_mean  
0        34174.7  
1        60289.8  
2        49442.5  
3        36366.9  
4          35494  
..           ...  
155        40537  
156        34244  
157      63910.4  
158      53403.1  
159      36962.3  

[160 rows x 9 columns]

In [3]:
dat['gender'].unique()

array(['Female', 'Male', 'Total', 'Unclassified'], dtype=object)

In [4]:
dat['year'] = [str(x)[:4] for x in dat['time_period']]
dat['gender_sign'] = ["\u2640" if i == "Female" else "\u2642" for i in dat['gender']]
dat['grade'] = ["Leadership teachers" if i == "Other Leadership teachers" else i for i in dat['grade']]

In [5]:
data = dat[["year", "gender", "gender_sign", "grade", "average_mean"]]\
       .loc[(dat['gender'].isin(['Female', 'Male'])) & (dat['grade'] != 'Total')].copy()
data.shape

(60, 5)

In [6]:
data['average_mean'] = pd.to_numeric(data['average_mean'])
data.dtypes

year             object
gender           object
gender_sign      object
grade            object
average_mean    float64
dtype: object

In [7]:
# check if the total entries are correct:
tcheck_gender = pd.pivot_table(data, 
                               values='average_mean', 
                               index=['grade'],
                               columns=['gender'], aggfunc=np.mean)

tcheck_gender 

gender                 Female      Male
grade                                  
Classroom teachers   34758.97  35700.61
Head teachers        64056.41  72011.81
Leadership teachers  50997.19  54859.20

In [8]:
# pay pattern in 2019
chart = alt.Chart(data.loc[data['year'] == '2019']).properties( width=50, height=200 ) 
text = chart.mark_text().encode(
    alt.X('year', title='Year', scale=alt.Scale(zero=False)),
    alt.Y('average_mean', title='Average Pay', scale=alt.Scale(zero=False)),
    color = "grade", text = "gender_sign", 
    column = alt.Column('grade', sort=['Classroom teachers', 'Other Leadership teachers']))
line = chart.mark_line().encode(
    alt.X('year', title='Year', scale=alt.Scale(zero=False)),
    alt.Y('average_mean', title='Average Pay'),
    color = "grade", strokeDash = 'year')
text 

alt.Chart(...)

In [9]:
# compare pay change in female teachers between 2010 and 2019
alt.Chart(data.loc[(data['gender'] == "Female") & (data['year'].isin(['2010', '2019']))]).mark_bar().encode(
    alt.X('year', title='Year', scale=alt.Scale(zero=False)),
    alt.Y('average_mean', title='Average Pay', scale=alt.Scale(zero=False)),
    color = 'grade',
    column = alt.Column('grade', sort=['Classroom teachers', 'Other Leadership teachers']))\
.properties( width=60, height=300 ) 

alt.Chart(...)

In [10]:
# complete time series for female teachers
alt.Chart(data.loc[data['gender'] == "Female"]).mark_line().encode(
    alt.X('year', title='Year', scale=alt.Scale(zero=False)),
    alt.Y('average_mean', title='Average Pay'),
    color = "grade")

alt.Chart(...)

In [11]:
points = alt.Chart(data, title = "Gender pay gaps among teachers in England").mark_text(size = 15).encode(
    alt.X('year', title='Year', 
          scale=alt.Scale(zero = False), 
          axis = alt.Axis(labelAngle = 0, values= [2010, 2013, 2016, 2019])),
    alt.Y('average_mean', title='Average Pay', axis = alt.Axis(titleAngle = 0, titleX=-90)),
    color = "grade", text = "gender_sign"
).properties( width=300, height=400 ) 

labels = alt.Chart(data).mark_text(align='left', dx = 10, dy = 10).encode(
    alt.X('year', aggregate='max'),
    alt.Y('average_mean', aggregate='max'),
    alt.Text('grade'),
    alt.Color('grade', legend=None))

points +labels

alt.LayerChart(...)